In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K

Using TensorFlow backend.


In [2]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [3]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [4]:
def DataSet(iFold,dup_tag,rand_tag):
    #
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    side_len  = math.ceil(math.sqrt(num_feature))
    num_class = 2
    #
    mat_ind_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/Mat_ind_F%d.txt") % (iFold)
    #
    mat_ind = file2array(mat_ind_path, "\t")
    
    if (int)(rand_tag):
        tmp_vec = np.zeros(num_feature)
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            tmp_vec[index] = int(mat_ind[r,c])
        index = [i for i in range(num_feature)]
        random.shuffle(index)
        tmp_vec = tmp_vec[index]
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            mat_ind[r,c] = int(tmp_vec[index])

    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, side_len, side_len, 1))
    Y_train = np.empty((nrow_train_dup, num_class))

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(side_len):
                for j in range(side_len):
                    if tmp_cnt < num_feature:
                        tmp_ind = mat_ind[i,j]
                        X_train[sam_count,i,j,0] = data_train[index,(int)(tmp_ind)]
                    else:
                        X_train[sam_count,i,j,0] = 0.5
                    tmp_cnt = tmp_cnt + 1
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, side_len, side_len, 1))
    Y_test  = np.empty((nrow_test, num_class))

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    tmp_ind = mat_ind[i,j]
                    X_test[index,i,j,0] = data_test[index,(int)(tmp_ind)]
                else:
                    X_test[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    return X_train,Y_train,X_test,Y_test

In [5]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [6]:
def neural(channel,height,width,classes):
    input_shape = (channel,height,width)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (height,width,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    model.add(Flatten())
    model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(classes,activation="softmax",name="fc2"))
    return model


In [7]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_label = model.predict_classes(sam_data, batch_size=32, verbose=1)
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    tmp_count = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    print(tmp_count)
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN,tmp_count
    

In [8]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_count_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]
all_count_test=[]

nRun = 10
side_len = 22
num_class = 2
for dup_tag in range(2):
    for ran_tag in range(2):
        for fold in range(nRun):
            iFold = fold + 1
            X_train,Y_train,X_test,Y_test = DataSet(iFold,dup_tag,ran_tag)
            print('X_train shape : ',X_train.shape)
            print('Y_train shape : ',Y_train.shape)
            print('X_test shape : ',X_test.shape)
            print('Y_test shape : ',Y_test.shape)
            #print(X_train[0])
            #print(Y_train[0,1])

            # # model
            #model = ResNet50(
            #    weights=None,
            #    classes=2
            #)

            ##
            model = neural(1,side_len,side_len,num_class)

            model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

            # # train
            model.fit(X_train, Y_train, epochs=100, batch_size=6)

            # # evaluate
            #model.evaluate(X_test, Y_test, batch_size=32)

            # #
            #models.append(model)

            # # save
            #save_path =("./my_resnet_model.h5_F%d") % (iFold)
            #model.save(save_path)

            #
            X_train,Y_train,X_test,Y_test = DataSet(iFold,0,ran_tag)
            train_N_TP,train_N_FP,train_N_TN,train_N_FN,train_count = get_statistics(model,X_train,Y_train)
            test_N_TP,test_N_FP,test_N_TN,test_N_FN,test_count = get_statistics(model,X_test,Y_test)
            all_TP_train.append(train_N_TP)
            all_FP_train.append(train_N_FP)
            all_TN_train.append(train_N_TN)
            all_FN_train.append(train_N_FN)
            all_count_train.append(train_count)
            all_TP_test.append(test_N_TP)
            all_FP_test.append(test_N_FP)
            all_TN_test.append(test_N_TN)
            all_FN_test.append(test_N_FN)
            all_count_test.append(test_count)


X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
1410/1410 [==============================] - 1s 915us/step - loss: 0.2725 - acc: 0.9291
Epoch 2/100
1410/1410 [==============================] - 1s 607us/step - loss: 0.2515 - acc: 0.9333
Epoch 3/100
1410/1410 [==============================] - 1s 636us/step - loss: 0.2503 - acc: 0.9333
Epoch 4/100
1410/1410 [==============================] - 1s 590us/step - loss: 0.2434 - acc: 0.9333
Epoch 5/100
1410/1410 [==============================] - 1s 609us/step - loss: 0.2399 - acc: 0.9333
Epoch 6/100
1410/1410 [==============================] - 1s 609us/step - loss: 0.2279 - acc: 0.9333
Epoch 7/100
1410/1410 [==============================] - 1s 607us/step - loss: 0.2258 - acc: 0.9333
Epoch 8/100
1410/1410 [==============================] - 1s 613us/step - loss: 0.2125 - acc: 0.9333
Epoch 9/100
1410/1410 [==============================] - 1s 613us/step - loss: 0

1410/1410 [==============================] - 1s 746us/step - loss: 0.0204 - acc: 0.9908
Epoch 81/100
1410/1410 [==============================] - 1s 807us/step - loss: 0.0070 - acc: 0.9986
Epoch 82/100
1410/1410 [==============================] - 1s 860us/step - loss: 0.0035 - acc: 0.9986
Epoch 83/100
1410/1410 [==============================] - 1s 720us/step - loss: 0.0145 - acc: 0.9957
Epoch 84/100
1410/1410 [==============================] - 1s 745us/step - loss: 0.0028 - acc: 1.0000
Epoch 85/100
1410/1410 [==============================] - 1s 725us/step - loss: 6.6757e-04 - acc: 1.0000
Epoch 86/100
1410/1410 [==============================] - 1s 720us/step - loss: 4.4113e-04 - acc: 1.0000
Epoch 87/100
1410/1410 [==============================] - 1s 740us/step - loss: 3.3953e-04 - acc: 1.0000
Epoch 88/100
1410/1410 [==============================] - 1s 726us/step - loss: 2.9322e-04 - acc: 1.0000
Epoch 89/100
1410/1410 [==============================] - 1s 746us/step - loss: 2.4501e-

Epoch 34/100
1410/1410 [==============================] - 1s 792us/step - loss: 0.0565 - acc: 0.9801
Epoch 35/100
1410/1410 [==============================] - 1s 769us/step - loss: 0.0534 - acc: 0.9816
Epoch 36/100
1410/1410 [==============================] - 1s 770us/step - loss: 0.0522 - acc: 0.9794
Epoch 37/100
1410/1410 [==============================] - 1s 757us/step - loss: 0.0499 - acc: 0.9801
Epoch 38/100
1410/1410 [==============================] - 1s 808us/step - loss: 0.0504 - acc: 0.9844
Epoch 39/100
1410/1410 [==============================] - 1s 827us/step - loss: 0.0454 - acc: 0.9851
Epoch 40/100
1410/1410 [==============================] - 1s 791us/step - loss: 0.0403 - acc: 0.9879
Epoch 41/100
1410/1410 [==============================] - 1s 826us/step - loss: 0.0347 - acc: 0.9865
Epoch 42/100
1410/1410 [==============================] - 1s 773us/step - loss: 0.0199 - acc: 0.9936
Epoch 43/100
1410/1410 [==============================] - 1s 795us/step - loss: 0.0183 - ac

D:\toolkits.win\Anaconda3-4.4.0\envs\dlwin36-CPU\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 22, 22, 1)
Y_train shape :  (1410, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
1410/1410 [==============================] - 1s 963us/step - loss: 0.2615 - acc: 0.9333
Epoch 2/100
1410/1410 [==============================] - 1s 836us/step - loss: 0.2505 - acc: 0.9340
Epoch 3/100
1410/1410 [==============================] - 1s 849us/step - loss: 0.2435 - acc: 0.9340
Epoch 4/100
1410/1410 [==============================] - 1s 865us/step - loss: 0.2361 - acc: 0.9340
Epoch 5/100
1410/1410 [==============================] - 1s 835us/step - loss: 0.2361 - acc: 0.9340
Epoch 6/100
1410/1410 [==============================] - 1s 817us/step - loss: 0.2342 - acc: 0.9340
Epoch 7/100
1410/1410 [==============================] - 1s 858us/step - loss: 0.2261 - acc: 0.9340
Epoch 8/100
1410/1410 [==============================] - 1s 828us/step - loss: 0.2141 - acc: 0.9340
Epoch 9/100
1410/1410 [==============================] - 1s 858us/step - loss: 0

...

2619/2619 [==============================] - 4s 1ms/step - loss: 1.1953e-07 - acc: 1.0000
Epoch 95/100
2619/2619 [==============================] - 4s 1ms/step - loss: 1.1946e-07 - acc: 1.0000
Epoch 96/100
2619/2619 [==============================] - 4s 1ms/step - loss: 1.1925e-07 - acc: 1.0000
Epoch 97/100
2619/2619 [==============================] - 4s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 98/100
2619/2619 [==============================] - 4s 1ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 99/100
2619/2619 [==============================] - 4s 1ms/step - loss: 1.2039e-07 - acc: 1.0000
Epoch 100/100
1410/1410 [==============================] - 1s 803us/step
[1317.   93.]
157/157 [==============================] - 0s 292us/step
[146.  11.]
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
2619/2619 [==============================] - 6s 2ms/step - loss: 0.6483 - acc: 0.6098
Epoch 2/100
2619/2619

2619/2619 [==============================] - 4s 2ms/step - loss: 0.0354 - acc: 0.9897
Epoch 50/100
2619/2619 [==============================] - 4s 2ms/step - loss: 0.0283 - acc: 0.9897
Epoch 51/100
2619/2619 [==============================] - 4s 2ms/step - loss: 0.0155 - acc: 0.9954
Epoch 52/100
2619/2619 [==============================] - 4s 1ms/step - loss: 0.0023 - acc: 1.0000
Epoch 53/100
2619/2619 [==============================] - 4s 2ms/step - loss: 5.1595e-04 - acc: 1.0000
Epoch 54/100
2619/2619 [==============================] - 4s 2ms/step - loss: 3.3289e-04 - acc: 1.0000
Epoch 55/100
2619/2619 [==============================] - 4s 2ms/step - loss: 2.4490e-04 - acc: 1.0000
Epoch 56/100
2619/2619 [==============================] - 4s 2ms/step - loss: 2.0328e-04 - acc: 1.0000
Epoch 57/100
2619/2619 [==============================] - 4s 2ms/step - loss: 1.5884e-04 - acc: 1.0000
Epoch 58/100
2619/2619 [==============================] - 4s 1ms/step - loss: 1.4050e-04 - acc: 1.0000

Epoch 27/100
2619/2619 [==============================] - 4s 1ms/step - loss: 8.7004e-05 - acc: 1.0000
Epoch 28/100
2619/2619 [==============================] - 4s 1ms/step - loss: 7.8243e-05 - acc: 1.0000
Epoch 29/100
2619/2619 [==============================] - 4s 1ms/step - loss: 5.2274e-05 - acc: 1.0000
Epoch 30/100
2619/2619 [==============================] - 4s 1ms/step - loss: 4.0645e-05 - acc: 1.0000
Epoch 31/100
2619/2619 [==============================] - 4s 1ms/step - loss: 3.2338e-05 - acc: 1.0000
Epoch 32/100
2619/2619 [==============================] - 4s 1ms/step - loss: 2.5202e-05 - acc: 1.0000
Epoch 33/100
2619/2619 [==============================] - 4s 1ms/step - loss: 2.1178e-05 - acc: 1.0000
Epoch 34/100
2619/2619 [==============================] - 4s 1ms/step - loss: 1.6086e-05 - acc: 1.0000
Epoch 35/100
2619/2619 [==============================] - 4s 1ms/step - loss: 0.1221 - acc: 0.9630
Epoch 36/100
2619/2619 [==============================] - 4s 1ms/step - loss:

Epoch 4/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.1870 - acc: 0.9423
Epoch 5/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.1373 - acc: 0.9605
Epoch 6/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.0863 - acc: 0.9772
Epoch 7/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.0747 - acc: 0.9780
Epoch 8/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.0371 - acc: 0.9913
Epoch 9/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.0260 - acc: 0.9947
Epoch 10/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.0381 - acc: 0.9875
Epoch 11/100
2633/2633 [==============================] - 4s 1ms/step - loss: 0.0211 - acc: 0.9935
Epoch 12/100
2633/2633 [==============================] - 3s 1ms/step - loss: 0.0367 - acc: 0.9875
Epoch 13/100
2633/2633 [==============================] - 3s 1ms/step - loss: 0.0154 - acc: 0.9951
Epoch 14/100
263

2633/2633 [==============================] - 4s 2ms/step - loss: 1.1968e-07 - acc: 1.0000
Epoch 62/100
2633/2633 [==============================] - 4s 2ms/step - loss: 1.1937e-07 - acc: 1.0000
Epoch 63/100
2633/2633 [==============================] - 4s 2ms/step - loss: 1.1950e-07 - acc: 1.0000
Epoch 64/100
2633/2633 [==============================] - 4s 2ms/step - loss: 1.1925e-07 - acc: 1.0000
Epoch 65/100
2633/2633 [==============================] - 4s 2ms/step - loss: 0.1389 - acc: 0.9723
Epoch 66/100
2633/2633 [==============================] - 4s 2ms/step - loss: 0.0608 - acc: 0.9806
Epoch 67/100
2633/2633 [==============================] - 4s 2ms/step - loss: 0.0068 - acc: 0.9985
Epoch 68/100
2633/2633 [==============================] - 4s 2ms/step - loss: 0.0303 - acc: 0.9871
Epoch 69/100
2633/2633 [==============================] - 4s 2ms/step - loss: 0.0057 - acc: 0.9977
Epoch 70/100
2633/2633 [==============================] - 4s 2ms/step - loss: 4.0817e-04 - acc: 1.0000
Epo

In [9]:
for dup_tag in range(2):
    for ran_tag in range(2):
        for iRun in range(nRun):
            print(all_TP_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FP_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_TN_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FN_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
        for iRun in range(nRun):
            print(all_TP_test[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FP_test[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_TN_test[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FN_test[dup_tag*2*nRun+ran_tag*nRun+iRun])

for dup_tag in range(2):
    for ran_tag in range(2):
        for iRun in range(nRun):
            print(all_count_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_count_test[dup_tag*2*nRun+ran_tag*nRun+iRun])


[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[1316.   94.]
[0. 0.]
[  94. 1316.]
[0. 0.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.   93.]
[0. 0.]
[  93. 1317.]
[0. 0.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.   93.]
[0. 0.]
[  93. 1317.]
[0. 0.]
[1317.   94.]
[0. 0.]
[  94. 1317.]
[0. 0.]
[1317.   94.]
[0. 0.]
[  94. 1317.]
[0. 0.]
[1317.   94.]
[0. 0.]
[  94. 1317.]
[0. 0.]
[141.   1.]
[9. 6.]
[  1. 141.]
[6. 9.]
[142.   1.]
[9. 5.]
[  1. 142.]
[5. 9.]
[144.   0.]
[10.  3.]
[  0. 144.]
[ 3. 10.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[138.   1.]
[10.  8.]
[  1. 138.]
[ 8. 10.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[141.   1.]
[10.  5.]
[  1. 141.]
[ 5. 10.]
[143.   1.]
[9. 3.]
[  1. 143.]
[3. 9.]
[140.   1.]
[9. 6.]
[  1. 140.]
[6. 9.]
[137.   2.]
[8. 9.]
[  2. 137.]
[9. 8.]
[875.  39.]
[ 55. 441.]
[ 39. 875.]
[441.  55.]
[928.  26.]
[ 68. 388.]
[ 26. 928.]
[388.  68.]
[1311.    4.]
[90.  5.]
[   4. 1311.

In [10]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))